In [1]:
import torch
import numpy as np
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

In [2]:
class DiabetesDataset(Dataset):
    def __init__(self):
        xy = np.loadtxt('diabetes.csv', delimiter=',', dtype=np.float32)
        self.len = xy.shape[0]
        self.x_data = torch.from_numpy(xy[:, 0:-1])
        self.y_data = torch.from_numpy(xy[:, [-1]])
        
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
    
    def __len__(self):
        return self.len

In [3]:
xy = np.loadtxt('diabetes.csv', delimiter=',', dtype=np.float32)
len = xy.shape[0]
x_data = torch.from_numpy(xy[:, 0:-1])
y_data = torch.from_numpy(xy[:, [-1]])


print(xy.shape)
print(xy.shape[0])
print(x_data.shape)
print(y_data.shape)

(759, 9)
759
torch.Size([759, 8])
torch.Size([759, 1])


In [4]:
dataset = DiabetesDataset()
# batch_size => 배치사이즈 (759/32 = 23개) 
# num_workers => 데이터로더 멀티쓰래딩(빠르게 데이터를 가져올 수 있음)
train_loader = DataLoader(dataset=dataset, batch_size=32, shuffle=True, num_workers=2)

In [5]:
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.l1 = torch.nn.Linear(8, 6)
        self.l2 = torch.nn.Linear(6, 4)
        self.l3 = torch.nn.Linear(4, 1)
        
        self.sigmoid = torch.nn.Sigmoid()
    
    def forward(self, x):
        out1 = self.sigmoid(self.l1(x))
        out2 = self.sigmoid(self.l2(out1))
        y_pred = self.sigmoid(self.l3(out2))
        
        return y_pred

In [6]:
model = Model()

In [7]:
criterion = torch.nn.BCELoss(reduction='mean')
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

In [8]:
for epoch in range(2):
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = Variable(inputs), Variable(labels)
        
        y_pred = model(inputs)

        loss = criterion(y_pred, labels)
        print(epoch, i, loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

0 0 0.6856139302253723
0 1 0.5194529891014099
0 2 0.6001556515693665
0 3 0.5773700475692749
0 4 0.5987153053283691
0 5 0.6674009561538696
0 6 0.6673120856285095
0 7 0.7111493349075317
0 8 0.6220303773880005
0 9 0.7091007232666016
0 10 0.6227344274520874
0 11 0.6011865139007568
0 12 0.7087043523788452
0 13 0.6226111650466919
0 14 0.7070028185844421
0 15 0.664315402507782
0 16 0.6028369069099426
0 17 0.6434855461120605
0 18 0.5407591462135315
0 19 0.6221234202384949
0 20 0.8176464438438416
0 21 0.7035054564476013
0 22 0.6245847344398499
0 23 0.6740406155586243
1 0 0.6439202427864075
1 1 0.6044740676879883
1 2 0.5640257000923157
1 3 0.66428542137146
1 4 0.623073399066925
1 5 0.767324686050415
1 6 0.7210488319396973
1 7 0.6432950496673584
1 8 0.5685293078422546
1 9 0.6047720909118652
1 10 0.5832666158676147
1 11 0.5816816091537476
1 12 0.5591020584106445
1 13 0.6660110950469971
1 14 0.6217886209487915
1 15 0.7093781232833862
1 16 0.7075766921043396
1 17 0.64382404088974
1 18 0.664336383342